In [ ]:
#pip install crewai

In [ ]:
#import sys
#print(sys.version)

In [ ]:
#pip install crewai_tools

In [1]:
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew, LLM
import yaml

In [ ]:
#llm = LLM(
 #   model="ollama/llama3.2",
  #  base_url="http://localhost:1"
#)

In [ ]:
from dotenv import load_dotenv
import os

os.environ["MISTRAL_API_KEY"] = "vb98"

In [ ]:
from openai import OpenAI


os.environ["OPENAI_API_KEY"] = "sk-proj-4Y"

In [5]:
llm = LLM(
    model="mistral/mistral-medium-2505",
    temperature=0.7,
    api_key=os.environ["MISTRAL_API_KEY"]
)

In [6]:
llm2 = LLM(
    model="openai/gpt-4o",
    api_key=os.environ["OPENAI_API_KEY"]
)

# **Loading Tasks and Agents YAML files**

In [7]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml',
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r', encoding= "utf-8") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [8]:
agents_config

{'profiler_agent': {'role': 'Learner Profiling and Insight Generation\n',
  'goal': 'Continuously collect and analyze personal (e.g., name, cultural background), behavioral (e.g., question frequency, complexity), and emotional data (e.g., sentiment from follow-up questions) for each learner. Build and maintain evolving learner profiles in learner_profile.json, reflecting cognitive, emotional, topic, and academic states. Detect language_preference from inputs (e.g., "fa" → "Farsi", "فارسی" → "Farsi") and normalize to full English names. If the learner profile data is incomplete, fill in missing fields with inferred or default values:\n      - academic_background: infer "STEM" or "Non-STEM" from topic or leave as "Unknown"\n      - emotional_state: default to "curious" if not provided\n      - language_style: default to ["casual"]\n      - learning_style: default to ["verbal"]\n',
  'backstory': 'You are part of a hierarchical CrewAI tutoring system, running first to provide learner prof

In [9]:
tasks_config

{'tasks': [{'name': 'Profile Learner',
   'description': 'Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., \'CNN\', \'history,...}\n',
   'expected_output': 'A JSON object named "profile_output" containing all the above fields with correct values and restricted options where specified. Example:\n  {\n    "p

# **Create Pydantic Models for Structured Output**

In [10]:
from crewai.tools import BaseTool

In [11]:
#!pip show crewai

In [12]:
import crewai

In [13]:
#!pip show crewai_tools

In [14]:
#from crewai_tools import FileReadTool

In [15]:
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

class LeadPersonalInfo(BaseModel):
    name: Optional[str] = Field(None, description="Learner's full name.")
    cultural_context: Optional[str] = Field("Other", description="Country-based cultural context.")
    language_preference: Optional[str] = Field("English", description="Preferred language.")
    academic_background: Optional[Literal["STEM", "Non-STEM", "Unknown"]] = Field(
        "Unknown", description="STEM or Non-STEM or Unknown"
    )
    emotional_state: Literal["motivated", "frustrated", "curious", "tired"] = Field(
        "curious", description="Defaults to 'curious'"  # Renamed from current_emotional
    )
    motivation_trend: Literal["increasing", "decreasing", "stable"] = Field(
        "stable", description="Motivation trend"
    )
    learning_style: Literal["verbal", "visual", "active", "reflective", "sequential", "global"] = Field(
        "verbal", description="Defaults to 'verbal'"
    )
    language_style: List[Literal["casual", "formal", "technical", "humor/sarcasm"]] = Field(
        default_factory=lambda: ["casual"], description="List of language styles"
    )
    learning_topic: Optional[str] = Field(
        None, description="Felder-Silverman related topic, no default to allow flexibility"
    )

class ProfileOutput(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Learner's full profile.")

class LeadScoringResult(BaseModel):
    profile_output: ProfileOutput = Field(..., description="Wrapper for learner profile.")

class LeadScoringResultList(BaseModel):
    results: List[LeadScoringResult] = Field(..., description="List of learner profiles to handle multiple tool calls.")

## **FileReadTool()**

In [16]:
# Markdown reader tool
#file_read_tool = FileReadTool(
 #   file_path='output_template.md',
  #  description='Reads the Template file to understand the expected output format.'
#)

## **JSONUpdateTool**

In [17]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Dict, Any, Union, Set
import json
import os
from datetime import datetime

class JSONUpdateInput(BaseModel):
    data: Dict[str, Any]

class JSONUpdateTool(BaseTool):
    name: str = "JSONUpdateTool"
    description: str = (
        "Updates nested fields in learner_profile.json with key-value pairs like 'learning_style.visual'. "
        "Allowed top-level keys: learning_style, cultural_context, language_style"
    )
    inputs: Optional[JSONUpdateInput] = Field(default_factory=lambda: JSONUpdateInput(data={}))
    profile_path: str = "learner_profile.json"
    log_path: str = "profile_update_log.json"
    allowed_keys: Set[str] = {"learning_style", "cultural_context", "language_style"}

    def _update_nested(self, d: dict, key_path: str, value: Any) -> (Any, Any):
        keys = key_path.split(".")
        top_key = keys[0]
        if top_key not in self.allowed_keys:
            return None, None

        ref = d
        for key in keys[:-1]:
            ref = ref.setdefault(key, {})
        
        old_value = ref.get(keys[-1], None)
        if old_value != value:
            ref[keys[-1]] = value
            return old_value, value
        else:
            return old_value, old_value

    def _load_json(self, path: str) -> dict:
        return json.load(open(path, "r", encoding="utf-8")) if os.path.exists(path) else {}

    def _save_json(self, path: str, data: dict):
        with open(path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)

    def _run(self, input_data: Union[str, dict, JSONUpdateInput]) -> str:
        try:
            if isinstance(input_data, str):
                input_model = JSONUpdateInput(data=json.loads(input_data))
            elif isinstance(input_data, dict):
                input_model = JSONUpdateInput(data=input_data)
            elif isinstance(input_data, JSONUpdateInput):
                input_model = input_data
            else:
                raise ValueError("Unsupported input type")

            profile = self._load_json(self.profile_path)
            change_log = {}
            actual_changes = False

            for key, value in input_model.data.items():
                old_val, new_val = self._update_nested(profile, key, value)
                if old_val != new_val:
                    change_log[key] = {"old": old_val, "new": new_val}
                    actual_changes = True

            if actual_changes:
                self._save_json(self.profile_path, profile)
                log = self._load_json(self.log_path)
                log[datetime.utcnow().isoformat()] = change_log
                self._save_json(self.log_path, log)
                return f"Updated profile with changes:\n{json.dumps(change_log, indent=2)}"
            else:
                return "No changes were needed in the profile."

        except json.JSONDecodeError as e:
            return f"Invalid JSON input: {e}"
        except Exception as e:
            return f"Error: {e}"


## FileWriteTool

In [18]:
import os
from typing import Union
from pydantic import BaseModel, Field
from crewai.tools import BaseTool

SAFE_DIR = os.path.abspath("workspace")

class FileWriteInput(BaseModel):
    filepath: str = Field(..., description="Path to the file to write to.")
    content: str = Field(..., description="Content to write into the file.")
    mode: str = Field("overwrite", description="Writing mode: 'overwrite' or 'append'")

def _is_safe_path(path: str) -> bool:
    abs_path = os.path.abspath(path)
    return abs_path.startswith(SAFE_DIR)

class FileWriteTool(BaseTool):
    name: str = "FileWriteTool"
    description: str = "Writes content to a file. Supports 'overwrite' or 'append' mode."

    def _run(self, input_data: Union[str, dict, FileWriteInput]) -> str:
        try:
            # Step 1: Parse input
            if isinstance(input_data, str):
                input_model = FileWriteInput.parse_raw(input_data)
            elif isinstance(input_data, dict):
                input_model = FileWriteInput(**input_data)
            elif isinstance(input_data, FileWriteInput):
                input_model = input_data
            else:
                raise ValueError("Unsupported input type")

            # Step 2: Check mode validity
            if input_model.mode not in {"overwrite", "append"}:
                return "Invalid mode. Use 'overwrite' or 'append'."

            # Step 3: Check safe path
            if not _is_safe_path(input_model.filepath):
                return f"Unsafe file path detected: {input_model.filepath}"

            write_mode = "w" if input_model.mode == "overwrite" else "a"

            # Step 4: If append mode, ensure content ends with newline
            content_to_write = input_model.content
            if write_mode == "a" and not content_to_write.endswith("\n"):
                content_to_write += "\n"

            # Step 5: Write to file
            with open(input_model.filepath, write_mode, encoding="utf-8") as f:
                written_bytes = f.write(content_to_write)

            return f"Successfully wrote {written_bytes} bytes to {input_model.filepath} in {input_model.mode} mode."

        except Exception as e:
            return f"Error writing to file: {e}"


# FileReadTool

In [19]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field, ValidationError, BeforeValidator
from typing import Optional, Any, Annotated, Union
import os

# Custom validator to handle string "None" and convert it to None
# This is useful if the LLM sometimes outputs "None" as a string for optional arguments
def validate_optional_int(v: Any) -> Optional[int]:
    """
    Validates an input value to be an optional integer.
    Converts string "None" to Python's None, attempts to convert strings to int,
    and raises an error for non-convertible strings.
    """
    if v is None or (isinstance(v, str) and v.lower() == 'none'):
        return None
    if isinstance(v, str):
        try:
            return int(v)
        except ValueError:
            raise ValueError(f"Input '{v}' cannot be converted to an integer.")
    return v

class ReadFileInput(BaseModel):
    """
    Input schema for the ReadFileTool.
    Defines the expected parameters for reading a file.
    """
    filepath: str = Field(..., description="The full path to the file to read.")
    # Annotated is used with BeforeValidator to preprocess input before Pydantic's main validation
    start_line: Annotated[Optional[int], BeforeValidator(validate_optional_int)] = Field(
        None, description="The line number (1-indexed) to start reading from. If None, reads from the beginning."
    )
    line_count: Annotated[Optional[int], BeforeValidator(validate_optional_int)] = Field(
        None, description="The number of lines to read. If None, reads until the end of the file or start_line is reached."
    )

class ReadFileTool(BaseTool):
    """
    A custom tool for CrewAI to read content from a file.
    It supports reading the entire file, or a specific range of lines.
    """
    name: str = "Read File Content"
    description: str = (
        "Reads content from a specified file. Use this to retrieve text from any file on the system. "
        "Always assume learner_profile.json is located in the current working directory, "
        "so use 'learner_profile.json' as the path."
    )
    # Assign the Pydantic input schema to the tool's args_schema
    args_schema: type[BaseModel] = ReadFileInput

    def _run(self, filepath: str, start_line: Optional[int] = None, line_count: Optional[int] = None) -> str:
        """
        Executes the file reading operation.
        Args are validated by Pydantic before this method is called.
        """
        try:
            if not os.path.exists(filepath):
                return f"Error: File not found at '{filepath}'."
            if not os.path.isfile(filepath):
                return f"Error: Path '{filepath}' is not a file."

            with open(filepath, 'r', encoding='utf-8') as f:
                lines = f.readlines()

            total_lines = len(lines)

            # Adjust start_line to be 0-indexed for Python list slicing
            actual_start_index = 0
            if start_line is not None:
                if start_line <= 0:
                    return f"Error: 'start_line' must be a positive integer (1-indexed)."
                actual_start_index = start_line - 1

            if actual_start_index >= total_lines:
                return f"Error: 'start_line' ({start_line}) is beyond the end of the file ({total_lines} lines)."

            # Determine the end index for slicing
            actual_end_index = total_lines
            if line_count is not None:
                if line_count <= 0:
                    return f"Error: 'line_count' must be a positive integer."
                actual_end_index = actual_start_index + line_count

            # Ensure the end index doesn't exceed the actual number of lines
            actual_end_index = min(actual_end_index, total_lines)

            content_lines = lines[actual_start_index:actual_end_index]

            if not content_lines and (start_line is not None or line_count is not None):
                 return f"No content found in the specified range (start_line: {start_line}, line_count: {line_count})."

            return "".join(content_lines)

        except Exception as e:
            # Catch any other unexpected errors during file operation
            return f"An unexpected error occurred while reading file '{filepath}': {e}"


my_read_tool = ReadFileTool()

# ImageGenerationTool

In [20]:
from pydantic import PrivateAttr

class ImageGenerationTool(BaseTool):
    name: str = "ImageGenerationTool"
    description: str = (
        "Generates an educational illustration for a given learning topic. "
        "Returns a URL to the generated image, suitable for embedding in markdown."
    )
    api_key: Optional[str] = None
    _client: Any = PrivateAttr()

    def __init__(self, api_key: str = None):
        super().__init__()
        # Use provided API key or fall back to environment variable
        self.api_key = api_key or os.environ.get("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OpenAI API key is required")
        self._client = OpenAI(api_key=self.api_key)

    def _run(self, learning_topic: str) -> str:
        """
        Generate an educational illustration for the given learning topic.
        Args:
            learning_topic (str): The topic for which to generate the illustration.
        Returns:
            str: The URL of the generated image.
        """
        prompt = (
            f"Create a clean, concept-focused educational illustration to explain the topic: '{learning_topic}'. "
            "It should be optimized for visual learners and help them intuitively grasp the core idea. "
            "Use symbols, diagrams, or metaphors. Avoid any text in the image. "
            "Use soft gradients, gentle lighting, and modern, minimalist visuals."
        )

        try:
            response = self._client.images.generate(
                model="dall-e-3",
                prompt=prompt,
                size="1024x1024",
                quality="standard",
                n=1,
            )
            return response.data[0].url
        except Exception as e:
            return f"Error generating image: {str(e)}"
        

        
        
        
image_tool = ImageGenerationTool()

# JSONSAVE

In [21]:
from typing import Type
from crewai.tools import BaseTool

class SingleJSONProfileInput(BaseModel):
    personal_info: Dict[str, Any]

class SingleJSONProfileTool(BaseTool):
    name: str = "SingleJSONProfileTool"
    description: str = "Appends a complete learner profile to profiles.json, preserving existing profiles in a list."
    args_schema: Type[BaseModel] = SingleJSONProfileInput
    profile_path: str = "profiles.json"
    def _run(self, personal_info: dict) -> str:
        try:
            profiles = []
            if os.path.exists(self.profile_path):
                with open(self.profile_path, "r", encoding="utf-8") as f:
                    try:
                        content = json.load(f)
                        if isinstance(content, list):
                            profiles = content
                        elif isinstance(content, dict):
                            profiles = [content]
                        else:
                            return f"Error: Invalid content in {self.profile_path}. Expected a list or dictionary."
                    except json.JSONDecodeError:
                        pass
            profiles.append(personal_info)
            with open(self.profile_path, "w", encoding="utf-8") as f:
                json.dump(profiles, f, indent=2)
            return f"Successfully appended profile to {self.profile_path}."
        except Exception as e:
            return f"Error saving profile: {e}"


# MarkdownToPdf

In [22]:
from pydantic import BaseModel
from crewai.tools import BaseTool
from typing import Type
import os
import subprocess

class MarkdownToPDFInput(BaseModel):
    markdown_content: str

class ConvertMarkdownToPDF(BaseTool):
    name: str = "convermarkdowntopdf"
    description: str = "A tool to convert Markdown content to a PDF file using mdpdf."
    args_schema: Type[BaseModel] = MarkdownToPDFInput

    def _run(self, markdown_content: str) -> str:
        temp_md_file = "temp_markdown.md"
        pdf_output_file = "final_lesson_bundle.pdf"

        # Write markdown content to a temporary file
        with open(temp_md_file, "w", encoding="utf-8") as f:
            f.write(markdown_content)

        # Run mdpdf to convert to PDF
        cmd = ['mdpdf', '--output', pdf_output_file, temp_md_file]
        try:
            subprocess.run(cmd, check=True)
        except subprocess.CalledProcessError as e:
            raise RuntimeError(f"PDF conversion failed: {e}")
        finally:
            # Clean up temporary file
            if os.path.exists(temp_md_file):
                os.remove(temp_md_file)

        return pdf_output_file

In [23]:
tool = ConvertMarkdownToPDF()
pdf_path = tool.run("# Sample markdown\nThis is test content.")
print(pdf_path)


Using Tool: convermarkdowntopdf
final_lesson_bundle.pdf


Logging to /home/897618/mdpdf.log
temp_markdown.md


In [24]:
pdf_tool = ConvertMarkdownToPDF()

# **RAG**

In [25]:
import json
with open('mango_dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)  # instead of json.loads(line)

    
json_str = json.dumps(data, indent=2)

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter

In [27]:
character_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                                   chunk_overlap=0)

chunks = character_splitter.split_text(json_str)

In [28]:
print(chunks[1])
print(f"\nTotal chunks: {len(chunks)}")

"id": "chatcmpl-8Iwe1LSdap9z6movNIJFcpXNQ9JnW_0_0_0_dist",
        "concept": "wedding",
        "culture": "Western culture",
        "assertion": "Brides wear white wedding gowns",
        "freq": 1
      },
      {
        "id": "chatcmpl-8Iwe7skxom4raCyz9DF1yWdDjkqd1_3_0_0_dist",
        "concept": "wedding",
        "culture": "Western cultures",
        "assertion": "White wedding dresses are traditional",
        "freq": 4
      },
      {
        "id": "chatcmpl-8IweBSzUIDOXpQw6HGBwrmoorLMu1_0_0_0_dist",
        "concept": "wedding",
        "culture": "Western countries",
        "assertion": "The bride wears a white dress.",
        "freq": 1
      },
      {
        "id": "chatcmpl-8IweNNS9hUfpCYBJFQdqtjHeUc4MA_0_0_0_dist",
        "concept": "wedding",
        "culture": "western countries",
        "assertion": "White wedding dresses are traditional",
        "freq": 3
      },
      {
        "id": "chatcmpl-8IwqE5dTLE7VkPKmL8TKXDyQZ8Bse_0_0_0_dist",

Total chunks: 201889

In [29]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_solit_text = []

for text in chunks:
    split_chunks = token_splitter.split_text(text)
    token_solit_text.extend(split_chunks)

Overriding of current TracerProvider is not allowed
2025-08-13 12:44:57.411734: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-13 12:44:57.411800: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-13 12:44:57.411820: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-13 12:44:57.602529: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropri

In [30]:
print (token_solit_text[1])
print(f"\nTotal chunks: {len(token_solit_text)}")

##dyp _ 0 _ 0 _ 0 _ dist ", " concept " : " wedding ", " culture " : " western cultures ", " assertion " : " white dress for the bride, traditional vows ", " freq " : 1 }, {

Total chunks: 395710


In [31]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_solit_text[10]]))

[array([-2.13429853e-02,  1.33399382e-01,  3.66154425e-02,  5.05943820e-02,
        3.85814086e-02,  2.14195624e-02, -9.19611286e-03, -7.40690231e-02,
        3.04757822e-02,  1.33120697e-02, -3.11695617e-02, -1.16469674e-01,
        1.41254039e-02,  1.25910267e-02,  7.58719370e-02,  6.56931773e-02,
       -3.80603932e-02, -2.99458741e-03, -8.92823935e-02, -2.15902962e-02,
       -9.76498239e-03, -4.56688330e-02,  8.67512599e-02, -1.76617224e-02,
       -1.51094217e-02, -3.21728848e-02,  2.67391913e-02,  7.07191378e-02,
       -4.67156898e-03,  1.13236597e-02,  3.68856192e-02,  7.45325387e-02,
        8.43261778e-02,  1.21715248e-01,  2.14801095e-02, -3.70536819e-02,
       -2.64314990e-02, -1.79712966e-04,  2.59540156e-02,  8.95774439e-02,
       -9.45717469e-03,  5.55281751e-02, -2.60929056e-02, -6.03872016e-02,
        6.04883581e-02,  1.03915883e-02, -4.46723960e-02,  3.78368869e-02,
       -8.05068761e-02,  6.05972596e-02, -4.01371643e-02,  2.30395868e-02,
       -3.34110633e-02, 

In [32]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.get_or_create_collection(
    name="mango_dataset", 
    embedding_function=embedding_function
)

ids = [str(i) for i in range(len(token_solit_text))]


batch_size = 5000
for i in range(0, len(token_solit_text), batch_size):
    batch_ids = ids[i:i+batch_size]
    batch_docs = token_solit_text[i:i+batch_size]
    chroma_collection.add(ids=batch_ids, documents=batch_docs)

print("Total documents in collection:", chroma_collection.count())

Total documents in collection: 395710


In [33]:
## Define as a tool(RAG tool)

In [42]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
import os
import chromadb
import requests

class RAGToolInput(BaseModel):
    query: str = Field(..., description="The query to search for in the knowledge base.")

class RAGTool(BaseTool):
    name: str = "RAG Chroma Research Tool"
    description: str = (
        "Retrieves culturally relevant examples from a knowledge base using Retrieval-Augmented Generation (RAG). "
        "Use this to fetch real-world examples or analogies based on the learner’s cultural context and learning topic."
        "Do not expect Mango to have direct technical matches. Instead, extract cultural traits (e.g. art, tradition, storytelling) from the learner’s country profile and creatively link them to the learning topic."

    )
    args_schema: type[BaseModel] = RAGToolInput
    api_key: str = Field(..., description="API key for Mistral")
    chroma_collection: chromadb.Collection = Field(..., description="ChromaDB collection for retrieving documents")

    def __init__(self, api_key: str, chroma_collection: chromadb.Collection):
        super().__init__(api_key=api_key, chroma_collection=chroma_collection)  # Pass all fields to parent class
        self.chroma_collection = chroma_collection  # Assign for clarity, though Pydantic handles it

    def _run(self, query: str) -> str:
        """
        Queries the ChromaDB collection to retrieve relevant documents and processes them with Mistral API.
        Returns a culturally relevant response or an error message if the query fails.
        """
        try:
            # Query ChromaDB for relevant documents
            results = self.chroma_collection.query(query_texts=[query], n_results=1)
            if not results['documents']:
                return "No relevant data found."
            
            # Use the rag function to process the retrieved documents
            retrieved_documents = results['documents']
            response = rag(query=query, retrieved_documents=retrieved_documents, api_key=self.api_key)
            return response
        except Exception as e:
            return f"Error querying knowledge base or processing with Mistral: {str(e)}"

# rag function 
def rag(query, retrieved_documents, api_key, model="mistral-large-latest"):
    url = "https://api.mistral.ai/v1/chat/completions"
    information = "\n\n".join(retrieved_documents)

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful AI tutor specializing in culturally relevant examples." 
                "When answering, if no direct cultural analogy is found in the documents,"
                "analyze the cultural traits in the Mango dataset (such as art, values, storytelling, traditions"
                "and use them creatively to build an analogy for the learning topic." 
                "For example, if the topic is CNN and the learner is from Iran, and Mango mentions traditional Persian art," 
                "you can compare CNN layers to layers of meaning in Persian miniature paintings or carpets."
            )
        },
        {
            "role": "user",
            "content": f"Question: {query}\n\nInformation:\n{information}"
        }
    ]

    payload = {
        "model": model,
        "messages": messages
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        result = response.json()
        return result["choices"][0]["message"]["content"]
    else:
        raise Exception(f"Request failed: {response.status_code} - {response.text}")

# Instantiate the tool with the chroma_collection and API key
rag_tool = RAGTool(
    api_key=os.environ.get("MISTRAL_API_KEY", "default-api-key"),  # Fallback if env var is missing
    chroma_collection=chroma_collection
)



## **Importing Tools**

In [35]:
print(agents_config["cultural_adapter_agent"].get("tools"))


['RAGTool']


In [36]:
# Creating Agents


tools_instances = [JSONUpdateTool(),FileWriteTool(),my_read_tool]
single_json_profile_tool = SingleJSONProfileTool()


profiler_agent = Agent(config=agents_config['profiler_agent'],
                      llm=llm, tools=tools_instances)

profile_saver_agnet = Agent(config=agents_config['profiler_agent'],
                      llm=llm, tools=[single_json_profile_tool])

content_generator_agent = Agent(config=agents_config['content_generator_agent'],
                      llm=llm,tools=[image_tool])


cultural_adapter_agent = Agent(config=agents_config['cultural_adapter_agent'],
                              llm=llm,tools=[rag_tool]
                              )


evaluator_agent = Agent(config=agents_config['evaluator_agent'],
                        llm=llm,tools=[ReadFileTool(),rag_tool]
                       )
pdf_creator_agent = Agent(config=agents_config['pdf_creator_agent'],
                        llm=llm,tools=[pdf_tool]
                       )

# Creating Tasks

profile_creator = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Profile Learner"),
    agent=profiler_agent
)


profile_json_creator = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Save Profile to JSON"),
    agent=profile_saver_agnet
)


generate_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Generate Content"),
    agent=content_generator_agent
) 

cultural_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Culturally Adapt Content"),
    agent=cultural_adapter_agent
) 

evaluator_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Evaluate Cultural Lesson"),
    agent=evaluator_agent
)


final_result = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Present Final Output"),
    agent=content_generator_agent
)

pdf_file = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "task_markdown_to_pdf"),
    agent=pdf_creator_agent
)

agents = [profiler_agent,profile_saver_agnet,content_generator_agent,cultural_adapter_agent,
          evaluator_agent,pdf_creator_agent]
tasks = [profile_creator, profile_json_creator, generate_content,
         cultural_content, evaluator_content, final_result,pdf_file]


# Testing

# First Example:
### Main LLM is MISTRAL
### Picture creator is GPT4o

In [118]:
## First Example:

## Visual(GPT4o)

In [37]:
#Sample Input

personal_data = {
    "personal_info": {
        "name": "Mojdeh Ghadami",
        "cultural_region": "Iran",
        "language_proficiency": "Fa",
        "learning_style": "Visual",
        "academic_background": "AI",
        "learning_topic": "algebra"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)

In [38]:
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)  

    
    
result = crew.kickoff()

print ('#####################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: I need to first read the current content of the learner_profile.json file to understand the existing data and iden



# Agent: Cultural Contextualization and Example Generator
## Thought: Thought: The initial query did not return any results. I will try querying with a more general context, using the continent "Middle East" instead of the specific country "Iran".
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"algebra Middle East\"}"
## Tool Output: 
Error querying knowledge base or processing with Mistral: sequence item 0: expected str instance, list found


# Agent: Cultural Contextualization and Example Generator
## Final Answer: 
```markdown
# مثال فرهنگی و سوال چند گزینه‌ای

## مثال فرهنگی

در بازارهای سنتی ایران، مانند بازار تهران، فروشندگان اغلب از جبر برای مدیریت معاملات روزانه خود استفاده می‌کنند. تصور کنید یک فروشنده فرش است و می‌خواهد قیمت فرش‌های خود را تعیین کند. او می‌داند که هزینه‌های ثابت او 500000 تومان است و هزینه هر متر مربع فرش 100000 تومان است. او می‌خواهد بداند که چقدر باید فرش بفروشد تا هزینه‌های خود را پوشش دهد.

در اینجا، او می‌تواند از جبر استفاده کند

Logging to /home/897618/mdpdf.log
temp_markdown.md
/home/897618/temp_markdown.md:21: https://oaidalleapiprodscus.blob.core.windows.net/private/org-HWO6wVL62f1pXZHd07pNYUU3/user-N0eug9QUFzaGpYBOL6IQvjOI/img-EB4B4nKvnNcrDKNL0k9suLHJ.png?st=2025-08-10T18%3A32%3A14Z&se=2025-08-10T20%3A32%3A14Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=7aed557a-269d-4dda-ab8b-c66e34024151&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-08-10T05%3A17%3A58Z&ske=2025-08-11T05%3A17%3A58Z&sks=b&skv=2024-08-04&sig=DOfoWPAl23/NHXPiW0CwglEZhvAz/%2BBJViNqo6mF7nw%3D: No such file or directory
/home/897618/temp_markdown.md:106: https://oaidalleapiprodscus.blob.core.windows.net/private/org-HWO6wVL62f1pXZHd07pNYUU3/user-N0eug9QUFzaGpYBOL6IQvjOI/img-EB4B4nKvnNcrDKNL0k9suLHJ.png?st=2025-08-10T18%3A32%3A14Z&se=2025-08-10T20%3A32%3A14Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=7aed557a-269d-4dda-ab8b-c66e34024151&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-08-10T05%3A17%3A58Z&ske=20



# Agent: PDF Converter',
## Thought: I need to convert the provided Markdown content into a PDF document. I will use the convermarkdowntopdf tool to accomplish this task.
## Using tool: convermarkdowntopdf
## Tool Input: 
"{\"markdown_content\": \"# \\u062f\\u0631\\u0633 \\u062c\\u0628\\u0631\\n\\n## \\u062a\\u0648\\u0636\\u06cc\\u062d\\n\\n\\u062c\\u0628\\u0631 \\u06cc\\u06a9\\u06cc \\u0627\\u0632 \\u0634\\u0627\\u062e\\u0647\\u200c\\u0647\\u0627\\u06cc \\u0627\\u0635\\u0644\\u06cc \\u0631\\u06cc\\u0627\\u0636\\u06cc\\u0627\\u062a \\u0627\\u0633\\u062a \\u06a9\\u0647 \\u0628\\u0627 \\u0645\\u0637\\u0627\\u0644\\u0639\\u0647 \\u0633\\u0627\\u062e\\u062a\\u0627\\u0631\\u060c \\u0631\\u0627\\u0628\\u0637\\u0647 \\u0648 \\u0645\\u0642\\u062f\\u0627\\u0631 \\u0633\\u0631 \\u0648 \\u06a9\\u0627\\u0631 \\u062f\\u0627\\u0631\\u062f. \\u062f\\u0631 \\u062c\\u0628\\u0631\\u060c \\u0645\\u0627 \\u0627\\u0632 \\u0646\\u0645\\u0627\\u062f\\u0647\\u0627 \\u0648 \\u062d\\u0631\\u0648\\u0641 \\u062



# Agent: PDF Converter',
## Final Answer: 
The final PDF document generated from the Markdown content is ready. Here is the complete content of the PDF:

```pdf
# درس جبر

## توضیح

جبر یکی از شاخه‌های اصلی ریاضیات است که با مطالعه ساختار، رابطه و مقدار سر و کار دارد. در جبر، ما از نمادها و حروف برای نمایش اعداد و مقادیر استفاده می‌کنیم. این به ما اجازه می‌دهد تا معادلات و فرمول‌ها را به صورت کلی بنویسیم و حل کنیم.

به عنوان مثال، معادله خطی ساده‌ای مانند \( ax + b = 0 \) را در نظر بگیرید. در اینجا، \( x \) یک متغیر است که ما باید مقدار آن را پیدا کنیم، و \( a \) و \( b \) اعدادی هستند که مقادیر آنها را می‌دانیم. حل این معادله به ما اجازه می‌دهد تا مقدار \( x \) را پیدا کنیم.

## مثال دنیای واقعی

تصور کنید که شما یک فروشنده سیب هستید و می‌خواهید قیمت سیب‌ها را تعیین کنید. شما می‌دانید که هزینه‌های ثابت شما 10000 تومان است و هزینه هر کیلوگرم سیب 5000 تومان است. شما می‌خواهید بدانید که چقدر باید سیب بفروشید تا هزینه‌های خود را پوشش دهید.

در اینجا، می‌توانید از جبر استفاده کنید. فرض ک

In [120]:
## second Example:

## only MISTRAL

In [39]:
# Sample input

import json

personal_data = {
    "personal_info": {
        "name": "Jerry Johanson",
        "cultural_context": "Denmark",
        "language_preference": "du",
        "motivation_trend": "stable",
        "learning_style": "texts",
        "learning_topic": "Ancient History"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)

In [40]:
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)



result = crew.kickoff()

print ('#####################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: I need to start by reading the current content of the learner_profile.json file to understand what information is 

# Agent: Cultural Contextualization and Example Generator
## Task: Retrieve and generate a culturally relevant real-world example and one multiple-choice question (MCQ) based on the learner’s cultural_context and the explanation content from lesson_content.md in the language_preference. Use the RAGTool to search the Mango dataset for content matching the learning_topic and cultural_context. Query should be a simple string like "{profile_output.personal_info.learning_topic} {profile_output.personal_info.cultural_context}". If no data is found, map the country to its continent and query again. Generate a globally understandable example if no data is found, with justification.



# Agent: Cultural Contextualization and Example Generator
## Thought: Thought: I need to retrieve culturally relevant examples and generate an MCQ based on the learner's cultural context (Denmark) and the learning topic (Ancient History). I will start by querying the RAG Chroma Research Tool with "Ancient History



# Agent: Educational Content Evaluator
## Final Answer: 
```markdown
# Cultural Evaluation

## Evaluation Result
Relevant

## PRACTICC Assessment

- **Personalization**: The example is somewhat tailored to the learner's cultural context (Denmark) by mentioning Viking settlements, but it could be more personalized.
- **Relevance**: The example is relevant to the learning topic of Ancient History and makes a good comparison between Viking settlements and ancient civilizations.
- **Accuracy**: The information provided about Viking settlements and their comparison to ancient civilizations is accurate.
- **Clarity**: The example is clear and easy to understand, using simple language and analogies.
- **Tone Alignment**: The tone is casual and aligns with the learner's language style preference.
- **Inclusivity**: The example is inclusive, considering the learner's cultural context and making a global comparison.
- **Confidence**: The example demonstrates confidence in the information prese

Logging to /home/897618/mdpdf.log
temp_markdown.md




# Agent: PDF Converter',
## Thought: To convert the provided Markdown content into a PDF document, I need to use the `convermarkdowntopdf` tool. This tool will take the Markdown content and generate a PDF file that preserves the formatting, structure, and embedded images.
First, I will combine all the provided Markdown sections into a single Markdown content string. Then, I will use the `convermarkdowntopdf` tool to convert this combined Markdown content into a PDF.
Here is the combined Markdown content:
markdown
# Ancient History Lesson
## Explanation
Hey there! Let's dive into Ancient History. Imagine we're talking about the really old days, like way before smartphones, cars, or even electricity. We're talking about thousands of years ago when people lived in cities like Athens, Rome, and Cairo.
Ancient History is all about understanding how people lived, what they believed in, and how they organized their societies. It's like looking at the roots of a big, old tree to understand h



# Agent: PDF Converter',
## Final Answer: 
The PDF file `final_lesson_bundle.pdf` has been successfully generated from the provided Markdown content. It includes the following sections:

1. **Ancient History Lesson**: An explanation of Ancient History, a real-world analogy comparing it to a family album, and an exploration of ancient civilizations like Egypt, Greece, and Rome.

2. **Cultural Examples**: An example of a Viking settlement in Denmark, a multiple-choice question about the pyramids, and an explanation of the correct answer.

3. **Cultural Evaluation**: An evaluation of the cultural example based on the PRACTICC assessment criteria, ensuring the content is personalized, relevant, accurate, clear, and aligns with the learner's preferences.

The PDF is ready for sharing or printing and accurately reflects the content and formatting of the original Markdown file.


#####################################
The PDF file `final_lesson_bundle.pdf` has been successfully generated fro

In [122]:
#Third Example:

In [41]:
# Sample input

import json

personal_data = {
    "personal_info": {
        "name": "Susan Smith",
        "cultural_context": "US",
        "language_preference": "us",
        "motivation_trend": "stable",
        "learning_style": "texts",
        "learning_topic": "international politics"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)
    
    
    
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)



result = crew.kickoff()

print ('#####################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: I need to start by reading the current content of the learner_profile.json file to understand what information is 



# Agent: Cultural Contextualization and Example Generator
## Thought: Thought: I need to retrieve culturally relevant examples and generate an MCQ based on the learner’s cultural context (US) and the learning topic (international politics). I will start by querying the RAG Chroma Research Tool with the specific cultural context.
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"international politics US\"}"
## Tool Output: 
Error querying knowledge base or processing with Mistral: sequence item 0: expected str instance, list found


# Agent: Cultural Contextualization and Example Generator
## Thought: Thought: Since the initial query resulted in an error, I will try a more general approach by using the continent as the cultural context. Since the US is in North America, I will query with "international politics North America".
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"international politics North America\"}"
## Tool Output: 
Error que



# Agent: Personalized Content Generator and Instructor
## Final Answer: 
```markdown
## Part 1: Lesson Content

# Lesson Content: International Politics

## Explanation

Hey there! Let's dive into the world of international politics. Imagine it as a big, global chess game where countries are the players. Each country has its own set of rules, goals, and strategies. They interact with each other through diplomacy, trade, alliances, and sometimes conflicts.

International politics is all about how countries make decisions and interact on a global scale. It involves understanding the relationships between nations, how they cooperate or compete, and how they handle issues like trade, security, and human rights. Think of it like a complex web where each strand represents a different type of interaction or relationship.

## Real-World Analogy

Let's use a simple analogy to make this clearer. Imagine you're at a big party with lots of different groups of people. Each group has its own inter

In [125]:

import json

personal_data = {
    "personal_info": {
        "name": "Susan Smith",
        "cultural_context": "America",
        "language_preference": "us",
        "motivation_trend": "stable",
        "learning_style": "verbs",
        "learning_topic": "international politics"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)
    
    
    
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)



result = crew.kickoff()

print ('#####################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: I need to analyze the provided learner data and update their profile in a JSON object. Since the actual learner da



# Agent: Cultural Contextualization and Example Generator
## Thought: Thought: I need to retrieve culturally relevant examples and generate an MCQ based on the learner’s cultural context (America) and the learning topic (international politics). I will use the RAG Chroma Research Tool to search for relevant information.
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"international politics America\"}"
## Tool Output: 
Error querying knowledge base or processing with Mistral: sequence item 0: expected str instance, list found


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: RAG Chroma Research Tool
Tool Arguments: {'query': {'description': 'The query to search for in the knowledge base.', 'type': 'str'}}
Tool Description: Retrieves culturally relevant examples from a knowledge base using Retrieval-Augmented Generation (RAG). Use this to fetch real-world examples or analogies based on the learner’



# Agent: Personalized Content Generator and Instructor
## Final Answer: 
```markdown
## Part 1: Lesson Content

# International Politics: A Simple Guide

## Explanation

International politics is like the rules and interactions between different countries. Imagine each country as a person, and international politics is how these "people" talk, argue, cooperate, and sometimes fight with each other. It involves things like making agreements (treaties), trading goods, and solving problems together.

Countries have their own interests and goals, just like people do. Sometimes they work together to achieve common goals, like protecting the environment or promoting peace. Other times, they might disagree and have conflicts. International politics is all about how countries manage these relationships.

## Real-World Analogy

Think of international politics like a big family reunion. Each family (country) has its own rules, traditions, and interests. At the reunion, families interact in diff

## using GPT4o

In [98]:
# Creating Agents


tools_instances = [JSONUpdateTool(),FileWriteTool(),my_read_tool]
single_json_profile_tool = SingleJSONProfileTool()


profiler_agent = Agent(config=agents_config['profiler_agent'],
                      llm=llm2, tools=tools_instances)

profile_saver_agnet = Agent(config=agents_config['profiler_agent'],
                      llm=llm2, tools=[single_json_profile_tool])

content_generator_agent = Agent(config=agents_config['content_generator_agent'],
                      llm=llm2,tools=[image_tool])


cultural_adapter_agent = Agent(config=agents_config['cultural_adapter_agent'],
                              llm=llm2,tools=[rag_tool]
                              )


evaluator_agent = Agent(config=agents_config['evaluator_agent'],
                        llm=llm2,tools=[ReadFileTool(),rag_tool]
                       )
pdf_creator_agent = Agent(config=agents_config['pdf_creator_agent'],
                        llm=llm2,tools=[pdf_tool]
                       )

# Creating Tasks

profile_creator = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Profile Learner"),
    agent=profiler_agent
)


profile_json_creator = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Save Profile to JSON"),
    agent=profile_saver_agnet
)


generate_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Generate Content"),
    agent=content_generator_agent
) 

cultural_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Culturally Adapt Content"),
    agent=cultural_adapter_agent
) 

evaluator_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Evaluate Cultural Lesson"),
    agent=evaluator_agent
)


final_result = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Present Final Output"),
    agent=content_generator_agent
)

pdf_file = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "task_markdown_to_pdf"),
    agent=pdf_creator_agent
)

agents = [profiler_agent,profile_saver_agnet,content_generator_agent,cultural_adapter_agent,
          evaluator_agent,pdf_creator_agent]
tasks = [profile_creator, profile_json_creator, generate_content,
         cultural_content, evaluator_content, final_result,pdf_file]


In [99]:
#Sample Input

personal_data = {
    "personal_info": {
        "name": "Mina Bagheri",
        "cultural_region": "Iran",
        "language_proficiency": "Fa",
        "learning_style": "Text",
        "academic_background": "AI",
        "learning_topic": "algebra"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)
    
    
    
    
    
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)



result = crew.kickoff()

print ('#####################################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: Thought: I need to read the existing learner profile data to determine what information is already present and wha

/home/897618/OODJupyterEnv/OOD-Jupyter-2025-07-02/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x




# Agent: Cultural Contextualization and Example Generator
## Thought: I should start by searching the Mango dataset for culturally relevant examples that connect the learning topic "algebra" with the cultural context "Iran". This will be done using the RAG Chroma Research Tool. If no relevant data is found for Iran specifically, I can broaden the search to include the Middle Eastern context.
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"algebra Iran\"}"
## Tool Output: 
Error querying knowledge base or processing with Mistral: sequence item 0: expected str instance, list found


# Agent: Cultural Contextualization and Example Generator
## Thought: It seems there was an issue with the previous action due to an error. However, I must continue to process the accurate action. Let's attempt again to fetch relevant examples or analogies from the Mango dataset for algebra in the context of Iran or the Middle East.
## Using tool: RAG Chroma Research Tool
## Tool Inpu

/home/897618/OODJupyterEnv/OOD-Jupyter-2025-07-02/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x




# Agent: Cultural Contextualization and Example Generator
## Thought: I encountered an error when querying the dataset. I will try broadening the context to the Middle East and see if that yields better results.
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"algebra Middle East\"}"
## Tool Output: 
Error querying knowledge base or processing with Mistral: sequence item 0: expected str instance, list found


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: RAG Chroma Research Tool
Tool Arguments: {'query': {'description': 'The query to search for in the knowledge base.', 'type': 'str'}}
Tool Description: Retrieves culturally relevant examples from a knowledge base using Retrieval-Augmented Generation (RAG). Use this to fetch real-world examples or analogies based on the learner’s cultural context and learning topic.Do not expect Mango to have direct technical matches. Instead, extract cultural tra

/home/897618/OODJupyterEnv/OOD-Jupyter-2025-07-02/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x




# Agent: Educational Content Evaluator
## Thought: Given the task, I need to check the relevance of the cultural examples using the PRACTICC rubric. First, I'll utilize the RAG Chroma Research Tool to check if the cultural content is sourced from the Mango dataset, considering Mina's cultural context (Iran).
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"algebra stories in Iranian culture\"}"
## Tool Output: 
Error querying knowledge base or processing with Mistral: sequence item 0: expected str instance, list found


# Agent: Educational Content Evaluator
## Final Answer: 
```markdown
## Cultural Evaluation

### Evaluation Result: Relevant

### PRACTICC Assessment:

- **Personalization**: The example directly refers to Iranian culture by incorporating a story from "One Thousand and One Nights," which connects algebra to personal and familiar cultural history for someone from Iran.
  
- **Relevance**: The content explicitly relates algebra to a well-known cult

Logging to /home/897618/mdpdf.log
temp_markdown.md




# Agent: PDF Converter',
## Thought: The task requires converting the provided Markdown content into a PDF document using the mdpdf library. I will utilize the convermarkdowntopdf tool to convert the Markdown content into a PDF, ensuring the preservation of formatting and structure.
## Using tool: convermarkdowntopdf
## Tool Input: 
"{\"markdown_content\": \"# \\u0645\\u062d\\u062a\\u0648\\u0627\\u06cc \\u062f\\u0631\\u0633: \\u062c\\u0628\\u0631\\n\\n## \\u062a\\u0648\\u0636\\u06cc\\u062d\\n\\n\\u062c\\u0628\\u0631 \\u06cc\\u06a9\\u06cc \\u0627\\u0632 \\u0634\\u0627\\u062e\\u0647\\u200c\\u0647\\u0627\\u06cc \\u0645\\u0647\\u0645 \\u0631\\u06cc\\u0627\\u0636\\u06cc\\u0627\\u062a \\u0627\\u0633\\u062a \\u06a9\\u0647 \\u0628\\u0647 \\u0645\\u0627 \\u0627\\u062c\\u0627\\u0632\\u0647 \\u0645\\u06cc\\u200c\\u062f\\u0647\\u062f \\u062a\\u0627 \\u0628\\u0627 \\u0627\\u0633\\u062a\\u0641\\u0627\\u062f\\u0647 \\u0627\\u0632 \\u0646\\u0645\\u0627\\u062f\\u0647\\u0627 \\u0648 \\u0627\\u0639\\u0



# Agent: PDF Converter',
## Final Answer: 
The Markdown content has been successfully converted into a PDF document titled "final_lesson_bundle.pdf", preserving the original formatting and structure. This PDF is now ready for sharing or printing.


#####################################################
The Markdown content has been successfully converted into a PDF document titled "final_lesson_bundle.pdf", preserving the original formatting and structure. This PDF is now ready for sharing or printing.


### Using MISTRAL as main LLM
### Using GPT4o as an Evaluator

In [42]:
# Creating Agents


tools_instances = [JSONUpdateTool(),FileWriteTool(),my_read_tool]
single_json_profile_tool = SingleJSONProfileTool()


profiler_agent = Agent(config=agents_config['profiler_agent'],
                      llm=llm, tools=tools_instances)

profile_saver_agnet = Agent(config=agents_config['profiler_agent'],
                      llm=llm, tools=[single_json_profile_tool])

content_generator_agent = Agent(config=agents_config['content_generator_agent'],
                      llm=llm,tools=[image_tool])


cultural_adapter_agent = Agent(config=agents_config['cultural_adapter_agent'],
                              llm=llm,tools=[rag_tool]
                              )


evaluator_agent = Agent(config=agents_config['evaluator_agent'],
                        llm=llm2,tools=[ReadFileTool(),rag_tool]
                       )
pdf_creator_agent = Agent(config=agents_config['pdf_creator_agent'],
                        llm=llm,tools=[pdf_tool]
                       )

# Creating Tasks

profile_creator = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Profile Learner"),
    agent=profiler_agent
)


profile_json_creator = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Save Profile to JSON"),
    agent=profile_saver_agnet
)


generate_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Generate Content"),
    agent=content_generator_agent
) 

cultural_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Culturally Adapt Content"),
    agent=cultural_adapter_agent
) 

evaluator_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Evaluate Cultural Lesson"),
    agent=evaluator_agent
)


final_result = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Present Final Output"),
    agent=content_generator_agent
)

pdf_file = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "task_markdown_to_pdf"),
    agent=pdf_creator_agent
)

agents = [profiler_agent,profile_saver_agnet,content_generator_agent,cultural_adapter_agent,
          evaluator_agent,pdf_creator_agent]
tasks = [profile_creator, profile_json_creator, generate_content,
         cultural_content, evaluator_content, final_result,pdf_file]


In [43]:
#Sample Input

personal_data = {
    "personal_info": {
        "name": "Kourosh Ghafari",
        "cultural_region": "Iran",
        "language_proficiency": "Fa",
        "learning_style": "Text",
        "academic_background": "History",
        "learning_topic": "Multi_agent AI system"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)
    
    
    
    
    
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)



result = crew.kickoff()

print ('#####################################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: First, I need to read the current content of the learner_profile.json file to understand the existing data and ide



# Agent: Educational Content Evaluator
## Thought: Action: RAG Chroma Research Tool
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"Iran marketplace multi-agent system example\"}"
## Tool Output: 
Error querying knowledge base or processing with Mistral: sequence item 0: expected str instance, list found


# Agent: Educational Content Evaluator
## Thought: Action: RAG Chroma Research Tool
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"Iranian culture multi-agent AI system example\"}"
## Tool Output: 
Error querying knowledge base or processing with Mistral: sequence item 0: expected str instance, list found


# Agent: Educational Content Evaluator
## Thought: Action: Read File Content
## Using tool: Read File Content
## Tool Input: 
"{\"filepath\": \"cultural_examples.md\", \"start_line\": null, \"line_count\": null}"
## Tool Output: 
Error: File not found at 'cultural_examples.md'.


You ONLY have access to the following tools, and shou

Logging to /home/897618/mdpdf.log
temp_markdown.md




# Agent: PDF Converter',
## Thought: Thought: I need to convert the provided Markdown content into a PDF document using the available tool.
## Using tool: convermarkdowntopdf
## Tool Input: 
"{\"markdown_content\": \"# \\u062f\\u0631\\u0633: \\u0633\\u06cc\\u0633\\u062a\\u0645 \\u0647\\u0648\\u0634 \\u0645\\u0635\\u0646\\u0648\\u0639\\u06cc \\u0686\\u0646\\u062f \\u0639\\u0627\\u0645\\u0644\\u06cc\\n\\n## \\u062a\\u0648\\u0636\\u06cc\\u062d\\n\\n\\u06cc\\u06a9 \\u0633\\u06cc\\u0633\\u062a\\u0645 \\u0647\\u0648\\u0634 \\u0645\\u0635\\u0646\\u0648\\u0639\\u06cc \\u0686\\u0646\\u062f \\u0639\\u0627\\u0645\\u0644\\u06cc (Multi-Agent AI System) \\u0645\\u062c\\u0645\\u0648\\u0639\\u0647\\u200c\\u0627\\u06cc \\u0627\\u0632 \\u0639\\u0627\\u0645\\u0644\\u200c\\u0647\\u0627\\u06cc \\u0647\\u0648\\u0634\\u0645\\u0646\\u062f \\u0627\\u0633\\u062a \\u06a9\\u0647 \\u0628\\u0627 \\u0647\\u0645 \\u06a9\\u0627\\u0631 \\u0645\\u06cc\\u200c\\u06a9\\u0646\\u0646\\u062f \\u062a\\u0627 \\u06cc\\u06a9 \\



# Agent: PDF Converter',
## Final Answer: 
final_lesson_bundle.pdf

```markdown
# درس: سیستم هوش مصنوعی چند عاملی

## توضیح

یک سیستم هوش مصنوعی چند عاملی (Multi-Agent AI System) مجموعه‌ای از عامل‌های هوشمند است که با هم کار می‌کنند تا یک هدف مشترک را دنبال کنند. هر عامل می‌تواند یک برنامه کامپیوتری باشد که قادر به درک محیط اطراف خود و انجام اقداماتی برای رسیدن به اهداف خود است. این عوامل می‌توانند با یکدیگر ارتباط برقرار کنند، همکاری کنند و حتی رقابت کنند تا بهترین نتیجه را به دست آورند.

## مثال واقعی

فرض کنید یک تیم فوتبال را در نظر بگیرید. هر بازیکن در تیم یک عامل هوشمند است که نقش خاص خود را دارد. آنها با هم کار می‌کنند تا گل بزنند و بازی را ببرند. هر بازیکن باید با توجه به موقعیت خود و موقعیت دیگر بازیکنان تصمیم بگیرد که چه کاری انجام دهد. آنها با هم ارتباط برقرار می‌کنند، پاس می‌دهند و استراتژی‌های مختلفی را برای رسیدن به هدف مشترک خود، یعنی بردن بازی، به کار می‌گیرند.

## نتیجه‌گیری

سیستم هوش مصنوعی چند عاملی مانند یک تیم فوتبال است که در آن هر عامل (بازیکن) نقش خاص خود را 

In [36]:
######################

In [44]:

import json

personal_data = {
    "personal_info": {
        "name": "Susan Smith",
        "cultural_context": "America",
        "language_preference": "us",
        "motivation_trend": "stable",
        "learning_style": "verbs",
        "learning_topic": "international politics"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)
    
    
    
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)



result = crew.kickoff()

print ('#####################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: I need to start by reading the current learner profile to understand what data is already available and what needs



# Agent: Cultural Contextualization and Example Generator
## Thought: Thought: I need to retrieve culturally relevant examples and generate an MCQ based on the learner's cultural context (America) and the learning topic (international politics). I will start by querying the RAG Chroma Research Tool with the learning topic and cultural context.
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"international politics America\"}"
## Tool Output: 
Error querying knowledge base or processing with Mistral: sequence item 0: expected str instance, list found


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: RAG Chroma Research Tool
Tool Arguments: {'query': {'description': 'The query to search for in the knowledge base.', 'type': 'str'}}
Tool Description: Retrieves culturally relevant examples from a knowledge base using Retrieval-Augmented Generation (RAG). Use this to fetch real-world examples or analogi



# Agent: Educational Content Evaluator
## Thought: Thought: Since I'm facing challenges obtaining the necessary cultural data via the RAG Chroma Research Tool, I should proceed with the information available to evaluate the provided cultural example in 'cultural_examples.md'. I'll assess the example against the PRACTICC criteria based on its content.
## Using tool: Read File Content
## Tool Input: 
"{\"filepath\": \"cultural_examples.md\"}"
## Tool Output: 
Error: File not found at 'cultural_examples.md'.


# Agent: Educational Content Evaluator
## Final Answer: 
Thought: Since I'm unable to access the file directly, I'll proceed by assessing the provided content that was shared in the task prompt regarding the cultural examples in international politics. I'll analyze it using the PRACTICC framework to determine its relevance and verify the source.

### Cultural Example Analysis
Given Example:
```markdown
## Cultural Example

Imagine the United Nations General Assembly as a grand sta

Logging to /home/897618/mdpdf.log
temp_markdown.md




# Agent: PDF Converter',
## Thought: I need to convert the provided Markdown content into a PDF document. Since I have the content directly, I can proceed with the conversion using the available tool.
## Using tool: convermarkdowntopdf
## Tool Input: 
"{\"markdown_content\": \"# Lesson Content: International Politics\\n\\n## Explanation\\n\\nInternational politics is like the rules and interactions that govern how countries behave and relate to each other on the global stage. It's about understanding how nations make decisions, form alliances, and sometimes have conflicts. Think of it as a big, complex game of chess where each country is a player trying to protect its interests and achieve its goals.\\n\\n## Real-World Analogy\\n\\nImagine you're at a big family reunion. Everyone at the reunion is part of the same family but has different interests, needs, and sometimes disagreements. Some family members are close and support each other, while others might have arguments or compete f

In [45]:
# Sample input

import json

personal_data = {
    "personal_info": {
        "name": "Susan Smith",
        "cultural_context": "US",
        "language_preference": "us",
        "motivation_trend": "stable",
        "learning_style": "texts",
        "learning_topic": "international politics"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)
    
    
    
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)



result = crew.kickoff()

print ('#####################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: I need to gather information about the learner to create or update their profile. Since I don't have the initial d



# Agent: Cultural Contextualization and Example Generator
## Thought: Thought: I need to retrieve culturally relevant examples and generate an MCQ based on the learner’s cultural context (US) and the learning topic (international politics). I will use the RAG Chroma Research Tool to search for relevant information.
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"international politics US\"}"
## Tool Output: 
Error querying knowledge base or processing with Mistral: sequence item 0: expected str instance, list found


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: RAG Chroma Research Tool
Tool Arguments: {'query': {'description': 'The query to search for in the knowledge base.', 'type': 'str'}}
Tool Description: Retrieves culturally relevant examples from a knowledge base using Retrieval-Augmented Generation (RAG). Use this to fetch real-world examples or analogies based on the learner’s cultural



# Agent: Personalized Content Generator and Instructor
## Final Answer: 
```markdown
## Part 1: Lesson Content

# Lesson Content: International Politics

## Explanation of International Politics

International politics is like the rules and interactions between different countries. It's about how nations make decisions, form friendships (or alliances), and sometimes have disagreements or conflicts. Just like people, countries have their own interests, needs, and ways of doing things. They work together on common goals, like protecting the environment or promoting trade, but they also compete for resources and influence.

## Real-World Analogy: The School Playground

Imagine international politics as a school playground. Each class (or country) has its own group of friends, its own games it likes to play, and its own rules. Sometimes, different classes come together to play a big game of soccer. They need to agree on the rules, like how long the game will last and what happens if some

Logging to /home/897618/mdpdf.log
temp_markdown.md




# Agent: PDF Converter',
## Thought: Thought: I need to convert the provided Markdown content into a PDF document. The content is already structured in Markdown format, so I can proceed with the conversion.
## Using tool: convermarkdowntopdf
## Tool Input: 
"{\"markdown_content\": \"# Lesson Content: International Politics\\n\\n## Explanation of International Politics\\n\\nInternational politics is like the rules and interactions between different countries. It's about how nations make decisions, form friendships (or alliances), and sometimes have disagreements or conflicts. Just like people, countries have their own interests, needs, and ways of doing things. They work together on common goals, like protecting the environment or promoting trade, but they also compete for resources and influence.\\n\\n## Real-World Analogy: The School Playground\\n\\nImagine international politics as a school playground. Each class (or country) has its own group of friends, its own games it likes to 

In [46]:
# Sample input

import json

personal_data = {
    "personal_info": {
        "name": "Jerry Johanson",
        "cultural_context": "Denmark",
        "language_preference": "du",
        "motivation_trend": "stable",
        "learning_style": "texts",
        "learning_topic": "Ancient History"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)
    

    
    
    
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)



result = crew.kickoff()

print ('#####################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: I need to analyze the provided learner data to update their profile in a JSON object. Since the user hasn't provid



# Agent: Cultural Contextualization and Example Generator
## Thought: Thought: Since the specific query did not work, I will attempt to broaden the search to the continent level. Denmark is part of Europe, so I will query with "Ancient History Europe".
## Using tool: RAG Chroma Research Tool
## Tool Input: 
"{\"query\": \"Ancient History Europe\"}"
## Tool Output: 
Error querying knowledge base or processing with Mistral: sequence item 0: expected str instance, list found


# Agent: Cultural Contextualization and Example Generator
## Final Answer: 
```markdown
# Cultural Examples

## Cultural Example
In Denemarken hebben we veel oude verhalen en legenden die ons helpen om de geschiedenis te begrijpen. Stel je voor dat je een oude Viking bent die op avontuur gaat. Net zoals de oude Grieken en Romeinen, hadden de Vikingen ook hun eigen cultuur en tradities. Ze bouwden grote schepen en verkenden nieuwe landen. Dit helpt ons te begrijpen hoe mensen in het verleden leefden en hoe hun avon



# Agent: PDF Converter',
## Final Answer: 
The PDF file has been successfully generated from the provided Markdown content. The content includes an explanation of ancient history, an analogy comparing ancient civilizations to the roots of a tree, a cultural example about Vikings, a multiple-choice question, and a cultural evaluation based on the PRACTICC assessment. The PDF is ready for sharing or printing.


#####################################
The PDF file has been successfully generated from the provided Markdown content. The content includes an explanation of ancient history, an analogy comparing ancient civilizations to the roots of a tree, a cultural example about Vikings, a multiple-choice question, and a cultural evaluation based on the PRACTICC assessment. The PDF is ready for sharing or printing.


## RAG with OPENAI

In [47]:
# Define the rag function
def rag(query, retrieved_documents, api_key, model="gpt-4o"):
    """
    Processes the query and retrieved documents using OpenAI GPT-4o.
    Returns the model's response.
    """
    if not api_key:
        raise ValueError("OpenAI API key is missing.")

    # Handle nested list structure from ChromaDB query
    if retrieved_documents and isinstance(retrieved_documents[0], list):
        information = "\n\n".join(retrieved_documents[0])  # Flatten the first inner list
    else:
        information = "\n\n".join(retrieved_documents)

    system_message = (
        "You are a helpful AI tutor specializing in culturally relevant examples. "
        "When answering, if no direct cultural analogy is found in the documents, "
        "analyze the cultural traits in the Mango dataset (such as art, values, storytelling, traditions) "
        "and use them creatively to build an analogy for the learning topic. "
        "For example, if the topic is CNN and the learner is from Iraq, and Mango mentions traditional Iraqi customs, "
        "you can compare CNN layers to the layered patterns in traditional Iraqi carpet weaving."
    )

    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": f"Question: {query}\n\nInformation:\n{information}"}
        ],
        "max_tokens": 1500,
        "temperature": 0.7
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result["choices"][0]["message"]["content"]
    else:
        raise Exception(f"OpenAI request failed: {response.status_code} - {response.text}")

# RAGTool class
class RAGToolInput(BaseModel):
    query: str = Field(..., description="The query to search for in the knowledge base.")

class RAGTool(BaseTool):
    name: str = "RAG Chroma Research Tool (OpenAI GPT-4o)"
    description: str = (
        "This tool leverages Retrieval-Augmented Generation (RAG), powered by OpenAI GPT-4o, to extract culturally relevant examples from a knowledge base rich with cultural information. "
        "The dataset contains detailed cultural insights, such as traditions, art, storytelling, and customs (e.g., how weddings are celebrated in Iraq). "
        "Use this tool to generate real-world examples or analogies tailored to the learner’s cultural context and learning topic. "
        "Note that the Mango dataset does not directly match technical topics (e.g., CNN). Instead, it extracts cultural traits (like art, traditions, or storytelling) from the learner’s country profile and creatively connects them to the learning topic. "
        "For example, it might compare CNN layers to the layered meanings in traditional Iraqi carpet weaving or Persian miniature paintings."
    )
    args_schema: type[BaseModel] = RAGToolInput
    api_key: str = Field(..., description="API key for OpenAI")
    chroma_collection: chromadb.Collection = Field(..., description="ChromaDB collection for retrieving documents")

    def __init__(self, api_key: str, chroma_collection: chromadb.Collection):
        super().__init__(api_key=api_key, chroma_collection=chroma_collection)
        self.chroma_collection = chroma_collection
        self.api_key = api_key

    def _run(self, query: str) -> str:
        """
        Queries the ChromaDB collection to retrieve relevant documents and processes them with OpenAI GPT-4o.
        Returns a culturally relevant response or an error message if the query fails.
        """
        try:
            results = self.chroma_collection.query(query_texts=[query], n_results=1)
            if not results['documents']:
                return "No relevant data found."

            retrieved_documents = results['documents']
            response = rag(
                query=query,
                retrieved_documents=retrieved_documents,
                api_key=self.api_key,
                model="gpt-4o"
            )
            return response
        except Exception as e:
            return f"Error querying knowledge base or processing with OpenAI: {str(e)}"

# Instantiate the tool
rag_tool = RAGTool(
    api_key=os.environ.get("OPENAI_API_KEY", None),
    chroma_collection=chroma_collection
)

# Example usage
response = rag_tool._run(query="Explain convolutional neural networks (CNN) using a cultural analogy for Iraq")
print("OpenAI Response:", response)

OpenAI Response: To explain convolutional neural networks (CNNs) using a cultural analogy relevant to Iraq, let's draw a parallel with the traditional art of Iraqi carpet weaving.

In Iraqi culture, carpet weaving is a revered craft, characterized by intricate patterns and layers of colors meticulously crafted by skilled artisans. Each layer and pattern contribute to the final, cohesive design, much like how a CNN processes an image.

1. **Layers in Carpet Weaving**: Just as an Iraqi carpet is built up layer by layer, each with its own distinct pattern or color, a CNN is composed of multiple layers. These layers, such as convolutional layers, pooling layers, and fully connected layers, work together to analyze different aspects of the input image.

2. **Patterns and Features**: When weaving a carpet, artisans focus on different patterns and motifs, which are akin to the features that a CNN learns to recognize in an image, such as edges, textures, and shapes. Early layers of a CNN might

In [48]:
# Creating Agents


tools_instances = [JSONUpdateTool(),FileWriteTool(),my_read_tool]
single_json_profile_tool = SingleJSONProfileTool()


profiler_agent = Agent(config=agents_config['profiler_agent'],
                      llm=llm2, tools=tools_instances)

profile_saver_agnet = Agent(config=agents_config['profiler_agent'],
                      llm=llm2, tools=[single_json_profile_tool])

content_generator_agent = Agent(config=agents_config['content_generator_agent'],
                      llm=llm2,tools=[image_tool])


cultural_adapter_agent = Agent(config=agents_config['cultural_adapter_agent'],
                              llm=llm2,tools=[rag_tool]
                              )


evaluator_agent = Agent(config=agents_config['evaluator_agent'],
                        llm=llm2,tools=[ReadFileTool(),rag_tool]
                       )
pdf_creator_agent = Agent(config=agents_config['pdf_creator_agent'],
                        llm=llm2,tools=[pdf_tool]
                       )

# Creating Tasks

profile_creator = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Profile Learner"),
    agent=profiler_agent
)


profile_json_creator = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Save Profile to JSON"),
    agent=profile_saver_agnet
)


generate_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Generate Content"),
    agent=content_generator_agent
) 

cultural_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Culturally Adapt Content"),
    agent=cultural_adapter_agent
) 

evaluator_content = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Evaluate Cultural Lesson"),
    agent=evaluator_agent
)


final_result = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "Present Final Output"),
    agent=content_generator_agent
)

pdf_file = Task(
    config=next(task for task in tasks_config["tasks"] if task["name"] == "task_markdown_to_pdf"),
    agent=pdf_creator_agent
)

agents = [profiler_agent,profile_saver_agnet,content_generator_agent,cultural_adapter_agent,
          evaluator_agent,pdf_creator_agent]
tasks = [profile_creator, profile_json_creator, generate_content,
         cultural_content, evaluator_content, final_result,pdf_file]




In [49]:
#Sample Input

personal_data = {
    "personal_info": {
        "name": "Mojdeh Ghadami",
        "cultural_region": "Iran",
        "language_proficiency": "Fa",
        "learning_style": "Visual",
        "academic_background": "AI",
        "learning_topic": "algebra"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)
    
    
    


In [50]:
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)  

    
    
result = crew.kickoff()

print ('#####################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: Thought: I need to read the learner_profile.json file to extract and analyze the current data for the learner in q



# Agent: Educational Content Evaluator
## Thought: Thought: I need to verify the source of the cultural example on algebra and ensure it meets the PRACTICC criteria using the tools available.
## Using tool: RAG Chroma Research Tool (OpenAI GPT-4o)
## Tool Input: 
"{\"query\": \"Al-Khwarizmi influence and analogy of Iranian carpets in algebra\"}"
## Tool Output: 
Al-Khwarizmi, a Persian mathematician, is often credited with significant contributions to the development of algebra. His work laid the foundation for modern algebraic concepts. To understand his influence through a cultural analogy, we can compare his contributions to the intricate patterns and craftsmanship found in Iranian carpets.

Just as a Persian carpet is meticulously crafted with layers of complex patterns and vibrant colors, each contributing to the overall beauty and functionality of the piece, algebraic concepts are built upon layers of logical reasoning and mathematical rules. Each "thread" or fundamental idea i

Logging to /home/897618/mdpdf.log
temp_markdown.md
Bad heading level.  More information:
Traceback (most recent call last):
  File "/home/897618/OODJupyterEnv/OOD-Jupyter-2025-07-02/lib/python3.11/site-packages/mdpdf/pdf_renderer.py", line 55, in __del__
    self.doc.set_toc(self.toc)
  File "/home/897618/OODJupyterEnv/OOD-Jupyter-2025-07-02/lib/python3.11/site-packages/fitz/utils.py", line 1325, in set_toc
    raise ValueError("hierarchy level of item 0 must be 1")
ValueError: hierarchy level of item 0 must be 1
[1, 'آموزش جبر', 1, 104.0]




# Agent: PDF Converter',
## Thought: To convert the provided Markdown content into a professionally formatted PDF document, I will use the `convermarkdowntopdf` tool.
## Using tool: convermarkdowntopdf
## Tool Input: 
"{\"markdown_content\": \"\\n## Part 1: Lesson Content\\n\\n# \\u0622\\u0645\\u0648\\u0632\\u0634 \\u062c\\u0628\\u0631\\n\\n## \\u062a\\u0648\\u0636\\u06cc\\u062d\\n\\n\\u062c\\u0628\\u0631 \\u0628\\u062e\\u0634\\u06cc \\u0627\\u0632 \\u0631\\u06cc\\u0627\\u0636\\u06cc\\u0627\\u062a \\u0627\\u0633\\u062a \\u06a9\\u0647 \\u0628\\u0627 \\u0627\\u0633\\u062a\\u0641\\u0627\\u062f\\u0647 \\u0627\\u0632 \\u0627\\u0639\\u062f\\u0627\\u062f \\u0648 \\u0645\\u062a\\u063a\\u06cc\\u0631\\u0647\\u0627 (\\u0645\\u0627\\u0646\\u0646\\u062f x \\u06cc\\u0627 y) \\u0645\\u0639\\u0627\\u062f\\u0644\\u0627\\u062a\\u06cc \\u0631\\u0627 \\u062d\\u0644 \\u0645\\u06cc\\u200c\\u06a9\\u0646\\u062f. \\u062f\\u0631 \\u062c\\u0628\\u0631\\u060c \\u0634\\u0645\\u0627 \\u06cc\\u0627\\u062f \\u0645\



# Agent: PDF Converter',
## Final Answer: 
The Markdown content has been successfully converted into a professionally formatted PDF document named `final_lesson_bundle.pdf`, preserving the original formatting, structure, and embedded elements for easy sharing or printing.


#####################################
The Markdown content has been successfully converted into a professionally formatted PDF document named `final_lesson_bundle.pdf`, preserving the original formatting, structure, and embedded elements for easy sharing or printing.


### Second

In [51]:
# Sample input

import json

personal_data = {
    "personal_info": {
        "name": "Jerry Johanson",
        "cultural_context": "Denmark",
        "language_preference": "du",
        "motivation_trend": "stable",
        "learning_style": "texts",
        "learning_topic": "Ancient History"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)

In [52]:
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)  

    
    
result = crew.kickoff()

print ('#####################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: Thought: I need to read the contents of the learner_profile.json file to analyze the available data and identify a



# Agent: Cultural Contextualization and Example Generator
## Thought: To find a culturally relevant example of ancient history in the context of Denmark, I will query the RAG Chroma Research Tool to see if there are any Danish-specific historical insights or analogies that align with the topic "Ancient History".
## Using tool: RAG Chroma Research Tool (OpenAI GPT-4o)
## Tool Input: 
"{\"query\": \"Ancient History Denmark\"}"
## Tool Output: 
When discussing ancient history in Denmark, we can delve into the rich tapestry of the Nordic Bronze Age and the Viking Age, which are periods filled with fascinating cultural artifacts and traditions. For instance, much like the layers of history built upon each other, ancient Danish society developed through various phases, each contributing to the cultural and social fabric of the region.

Imagine the intricate designs found in ancient Danish jewelry and artifacts from the Bronze Age. These designs often involve complex patterns and motifs tha

Logging to /home/897618/mdpdf.log
temp_markdown.md




# Agent: PDF Converter',
## Thought: Thought: I need to convert the provided Markdown content into a PDF document while maintaining its formatting and ensuring that the content is accurately represented. I will use the available tool to perform this conversion.
## Using tool: convermarkdowntopdf
## Tool Input: 
"{\"markdown_content\": \"# Ancient History Lesson\\n\\n## Forklaring til Gamle Historie\\n\\nHej Jerry! Velkommen til en rejse tilbage i tiden. N\\u00e5r vi taler om gammel historie, taler vi om de tidspunkter i verden, hvor civilisationer f\\u00f8rst begyndte at opst\\u00e5 og mennesker begyndte at opfinde ting som sprog, skrift og selv politik.\\n\\nT\\u00e6nk p\\u00e5 det som at g\\u00e5 igennem et gammelt bibliotek, hvor hver bog er en ny civilisation med sine egne eventyr og mysterier at opdage.\\n\\n## Analogi: Gamle Historie er som et Antikt Museum\\n\\nForestil dig, at gammel historie er som et antikt museum. Hver civilisation er som en gammel maleriudstilling eller s

### Third

In [53]:
# Sample input

import json

personal_data = {
    "personal_info": {
        "name": "Susan Smith",
        "cultural_context": "US",
        "language_preference": "us",
        "motivation_trend": "stable",
        "learning_style": "texts",
        "learning_topic": "international politics"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)
    
    
    
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)



result = crew.kickoff()

print ('#####################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: Thought: I should check if there is any existing data in learner_profile.json, which might provide me with some of



# Agent: Cultural Contextualization and Example Generator
## Thought: Thought: I need to search for culturally relevant examples related to "international politics" in the context of the US. I will use the RAG Chroma Research Tool to find information from the Mango dataset.
## Using tool: RAG Chroma Research Tool (OpenAI GPT-4o)
## Tool Input: 
"{\"query\": \"international politics US\"}"
## Tool Output: 
In the context of international politics and the United States, let's consider the analogy of jazz music, a key aspect of American culture. Jazz is characterized by its improvisational nature, diverse influences, and the ability to adapt and evolve. Similarly, international politics involving the US often requires adaptability, quick thinking, and the integration of various global influences and perspectives.

Just as jazz musicians must listen to one another and adjust their playing to the dynamics of the group, US diplomats and politicians must be attuned to the shifting dynamics 



# Agent: Personalized Content Generator and Instructor
## Final Answer: 
```markdown
## Part 1: Lesson Content

# Understanding International Politics

### Explanation

International politics involves the interactions between countries on the global stage. These interactions can be cooperative, such as forming alliances and trade agreements, or competitive, as seen in conflicts and diplomatic negotiations. Various elements, such as economics, cultural exchanges, and military capabilities, play crucial roles in shaping these relationships.

### Real-World Analogy

Imagine the world as a neighborhood where each house represents a different country. In this neighborhood, each family (or country) has its interests, traditions, resources, and security concerns. Some families get along well and often share resources, support each other's events, or provide help in times of need. These are akin to countries forming alliances or entering trade agreements.

However, like in any neighborhood, 

Logging to /home/897618/mdpdf.log
temp_markdown.md
Bad heading level.  More information:
Traceback (most recent call last):
  File "/home/897618/OODJupyterEnv/OOD-Jupyter-2025-07-02/lib/python3.11/site-packages/mdpdf/pdf_renderer.py", line 55, in __del__
    self.doc.set_toc(self.toc)
  File "/home/897618/OODJupyterEnv/OOD-Jupyter-2025-07-02/lib/python3.11/site-packages/fitz/utils.py", line 1325, in set_toc
    raise ValueError("hierarchy level of item 0 must be 1")
ValueError: hierarchy level of item 0 must be 1
[1, 'Understanding International Politics', 1, 104.0]




# Agent: PDF Converter',
## Thought: Given the task at hand, I need to convert the Markdown content provided into a PDF document using the mdpdf library. This requires accurately converting the structure, formatting, and embedded elements as specified in the Markdown text.
## Using tool: convermarkdowntopdf
## Tool Input: 
"{\"markdown_content\": \"\\n## Part 1: Lesson Content\\n\\n# Understanding International Politics\\n\\n### Explanation\\n\\nInternational politics involves the interactions between countries on the global stage. These interactions can be cooperative, such as forming alliances and trade agreements, or competitive, as seen in conflicts and diplomatic negotiations. Various elements, such as economics, cultural exchanges, and military capabilities, play crucial roles in shaping these relationships.\\n\\n### Real-World Analogy\\n\\nImagine the world as a neighborhood where each house represents a different country. In this neighborhood, each family (or country) has its

### Fourth

In [54]:
#Sample Input

personal_data = {
    "personal_info": {
        "name": "Kourosh Ghafari",
        "cultural_region": "Iran",
        "language_proficiency": "Fa",
        "learning_style": "Text",
        "academic_background": "History",
        "learning_topic": "Multi_agent AI system"
    }
}

with open('learner_profile.json', 'w') as file:
    json.dump(personal_data, file, indent=4)
    
    
    
    
    
crew = Crew(
    agents = agents,
    tasks = tasks,
    verbose= False)



result = crew.kickoff()

print ('#####################################################')
print (result)

# Agent: Learner Profiling and Insight Generation
## Task: Analyze the provided learner data to update their profile in a JSON object. If any expected fields are missing (e.g., emotional_state, academic_background, or language_style), use intelligent defaults based on context or general heuristics. - name: {e.g., "Mojdeh Ghadami"} - cultural_context: {e.g., "Iran", "USA",...} - language_preference: {e.g., "Farsi", "English", "Arabic", "French", "Spanish",} - academic_background: {"STEM", "Non-STEM", or "Unknown"} - emotional_state: {e.g., motivated, frustrated, curious, tired} - motivation_trend: {"increasing", "decreasing", or "stable"} - learning_style: {based on Felder-Silverman Learning Styles Model} - language_style: {casual, formal, technical, humor/sarcasm} - learning_topic: {e.g., 'CNN', 'history,...}



# Agent: Learner Profiling and Insight Generation
## Thought: Thought: I need to first access the existing learner profile data from the 'learner_profile.json' to analyze it an

# Agent: Cultural Contextualization and Example Generator
## Task: Retrieve and generate a culturally relevant real-world example and one multiple-choice question (MCQ) based on the learner’s cultural_context and the explanation content from lesson_content.md in the language_preference. Use the RAGTool to search the Mango dataset for content matching the learning_topic and cultural_context. Query should be a simple string like "{profile_output.personal_info.learning_topic} {profile_output.personal_info.cultural_context}". If no data is found, map the country to its continent and query again. Generate a globally understandable example if no data is found, with justification.



# Agent: Cultural Contextualization and Example Generator
## Thought: Thought: I need to use the RAG Chroma Research Tool to find culturally relevant examples related to the "Multi-agent AI system" within the context of Iran. This will help me generate a culturally resonant analogy and a multiple-choice question 



# Agent: Educational Content Evaluator
## Thought: Since there was an error locating the file 'cultural_examples.md', I'll verify if it's in another expected location. However, instead of trying random file paths, I'll continue using one tool correctly to double-check the availability in the default directory.
## Using tool: Read File Content
## Tool Input: 
"{\"filepath\": \"cultural_examples.md\"}"
## Tool Output: 
Error: File not found at 'cultural_examples.md'.


# Agent: Educational Content Evaluator
## Thought: Given that 'cultural_examples.md' cannot be located, I'll proceed to evaluate the examples using the RAG Chroma Research Tool with direct cultural insights pertaining to Iran, which aligns with the user 'Kourosh Ghafari's profile. I'll validate if the examples meet the cultural fidelity expected.
Let's try using the RAG Chroma Research Tool next.
## Using tool: RAG Chroma Research Tool (OpenAI GPT-4o)
## Tool Input: 
"{\"query\": \"Iranian art storytelling and multi-agen

Logging to /home/897618/mdpdf.log
temp_markdown.md




# Agent: PDF Converter',
## Thought: To convert the given Markdown content into a PDF document using the mdpdf library, I will use the `convermarkdowntopdf` tool. The Markdown content includes sections on multi-agent AI systems, cultural examples and questions, and an evaluation section. This content is structured for a learner named Kourosh with a verbal learning style. I will execute the conversion now.
## Using tool: convermarkdowntopdf
## Tool Input: 
"{\"markdown_content\": \"# \\u0622\\u0645\\u0648\\u0632\\u0634 \\u0633\\u06cc\\u0633\\u062a\\u0645\\u200c\\u0647\\u0627\\u06cc \\u0647\\u0648\\u0634 \\u0645\\u0635\\u0646\\u0648\\u0639\\u06cc \\u0686\\u0646\\u062f\\u0639\\u0627\\u0645\\u0644\\u0647\\n\\n## \\u062a\\u0648\\u0636\\u06cc\\u062d\\n\\n\\u0633\\u06cc\\u0633\\u062a\\u0645\\u200c\\u0647\\u0627\\u06cc \\u0647\\u0648\\u0634 \\u0645\\u0635\\u0646\\u0648\\u0639\\u06cc \\u0686\\u0646\\u062f\\u0639\\u0627\\u0645\\u0644\\u0647 (Multi-agent AI Systems) \\u0634\\u0627\\u0645\\u0644



# Agent: PDF Converter',
## Final Answer: 
The Markdown content has been successfully converted into a professionally formatted PDF document named `final_lesson_bundle.pdf`. This PDF accurately reflects the structure, formatting, and collaborative concepts presented in the original Markdown content. It is ready for sharing or printing.


#####################################################
The Markdown content has been successfully converted into a professionally formatted PDF document named `final_lesson_bundle.pdf`. This PDF accurately reflects the structure, formatting, and collaborative concepts presented in the original Markdown content. It is ready for sharing or printing.


In [43]:
## To compare the output of rags

In [44]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
import os
import chromadb
import requests

class RAGToolInput(BaseModel):
    query: str = Field(..., description="The query to search for in the knowledge base.")


class RAGTool(BaseTool):
    name: str = "RAG Chroma Research Tool"
    description: str = (
        "Retrieves culturally relevant examples from a knowledge base using Retrieval-Augmented Generation (RAG). "
        "Do not expect Mango to have direct technical matches. Instead, extract cultural traits "
        "(e.g. art, tradition, storytelling) from the learner’s country profile and creatively link them to the learning topic."
    )
    args_schema: type[BaseModel] = RAGToolInput
    api_key: str = Field(..., description="API key for Mistral")
    chroma_collection: chromadb.Collection = Field(..., description="ChromaDB collection for retrieving documents")

    def __init__(self, api_key: str, chroma_collection: chromadb.Collection):
        super().__init__(api_key=api_key, chroma_collection=chroma_collection)
        self.api_key = api_key
        self.chroma_collection = chroma_collection

    def _run(self, query: str) -> str:
        try:
            results = self.chroma_collection.query(query_texts=[query], n_results=3)
            if not results.get('documents'):
                return "No relevant data found."

            # Flatten the list of lists
            retrieved_documents = [
                doc for sublist in results['documents'] for doc in sublist
                if isinstance(doc, str)
            ]

            if not retrieved_documents:
                return "No relevant text documents found."

            return rag(query=query, retrieved_documents=retrieved_documents, api_key=self.api_key)

        except Exception as e:
            return f"Error querying knowledge base or processing with Mistral: {str(e)}"


# ===================== 5. Define RAG Function =====================
def rag(query, retrieved_documents, api_key, model="mistral-large-latest"):
    url = "https://api.mistral.ai/v1/chat/completions"
    information = "\n\n".join(retrieved_documents)

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful AI tutor specializing in culturally relevant examples. "
                "When answering, if no direct cultural analogy is found in the documents, "
                "analyze the cultural traits in the Mango dataset (such as art, values, storytelling, traditions) "
                "and use them creatively to build an analogy for the learning topic."
            )
        },
        {
            "role": "user",
            "content": f"Question: {query}\n\nInformation:\n{information}"
        }
    ]

    payload = {"model": model, "messages": messages}
    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        result = response.json()
        return result["choices"][0]["message"]["content"]
    else:
        raise Exception(f"Request failed: {response.status_code} - {response.text}")


# ===================== 6. Run Example =====================
rag_tool = RAGTool(
    api_key=os.environ.get("MISTRAL_API_KEY", "default-api-key"),
    chroma_collection=chroma_collection
)

response = rag_tool._run(query="Explain convolutional neural networks (CNN) using a cultural analogy for Iraq")
print("MISTRAL Response:", response)

MISTRAL Response: To explain **Convolutional Neural Networks (CNNs)** using a culturally relevant analogy for **Iraq**, let’s draw inspiration from **traditional Iraqi carpet weaving**—a deeply rooted art form that reflects precision, layered patterns, and the transmission of meaning through intricate design. Here’s how the analogy breaks down:

---

### **CNNs as Iraqi Carpet Weaving: Layer by Layer, Pattern by Pattern**
1. **Input Image = The Blank Loom (النسيج الفارغ)**
   - Just as a weaver starts with a blank loom (the foundation for a carpet), a CNN begins with an **input image**—a grid of pixels (like the empty threads waiting to be woven).
   - Example: Imagine the input is a photograph of the **Ziggurat of Ur**—the weaver (or CNN) must "understand" its shapes, textures, and symbols to recreate it in thread.

2. **Convolutional Layers = The Weaver’s Motifs (الزخارف التقليدية)**
   - In Iraqi carpets, weavers use **repeated motifs** (like geometric stars, floral patterns, or cal

In [103]:
#!pip install pymupdf==1.22.3
#!pip install mdpdf


In [105]:
#!pip install gradio

In [42]:
#import gradio as gr

In [1]:
# Gradio processing function
#def process_json_file(json_file):
 #   try:
        # Save uploaded JSON file as learner_profile.json
  #      with open("learner_profile.json", "wb") as f:
   #         f.write(json_file.read())
    #    
     #   # Load and validate JSON
      #  with open("learner_profile.json", "r", encoding="utf-8") as f:
       #     personal_data = json.load(f)
        #
        ## Validate JSON structure
        #if not isinstance(personal_data, dict) or "personal_info" not in personal_data:
          #  return "Error: Invalid JSON structure. Expected 'personal_info' key."

        # Run CrewAI workflow
        #crew = Crew(agents=[profiler_agent, json_saver_agent, content_generator_agent, cultural_adapter_agent, evaluator_agent, pdf_creator_agent], tasks=tasks, verbose=False)
        #result = crew.kickoff()

        # Check if PDF was generated
        #pdf_path = "final_lesson_bundle.pdf"
        #if os.path.exists(pdf_path):
         #   return pdf_path, "PDF generated successfully. Download it below."
        #else:
         #   return None, "Error: PDF generation failed."
    #except json.JSONDecodeError:
     #   return None, "Error: Invalid JSON file."
    #except Exception as e:
     #   return None, f"Error: {str(e)}"

# Create Gradio interface
#iface = gr.Interface(
 #   fn=process_json_file,
  #  inputs=gr.File(label="Upload Learner Profile JSON"),
   ## outputs=[
     #   gr.File(label="Download Generated PDF"),
      #  gr.Textbox(label="Status")
    #],
    #title="CrewAI Tutoring System",
    #description="Upload a learner profile JSON file to generate a personalized lesson PDF."
#)

# Launch the interface
#iface.launch()